In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import math
import pickle

In [2]:
df = pd.read_csv('fifa21_train.csv')
#print(df.shape)
#print(df.dtypes)
print(df.isna().sum())

ID              0
Name            0
Age             0
Nationality     0
Club           18
               ..
CB              0
RCB             0
RB              0
GK              0
OVA             0
Length: 101, dtype: int64


In [3]:
df.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [4]:
numerical_cols = df._get_numeric_data()
categorical_cols = df.select_dtypes('object')

In [5]:
numerical_cols

,ID,Age,Growth,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,...,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,184383,26,1,258,54,47,43,70,44.0,286,...,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,...,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,...,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,...,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,...,15,1664,360,66,44,60,64,60,66,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,25,5,224,62,24,51,59,28.0,248,...,14,1589,347,75,28,56,59,60,69,65
11697,214680,27,4,84,12,14,13,29,16.0,96,...,72,1123,387,72,67,66,72,40,70,70
11698,221489,22,8,286,64,66,51,64,41.0,291,...,16,1603,337,76,60,60,72,26,43,67
11699,146717,26,7,254,64,66,51,73,NaN,284,...,24,1429,400,72,63,70,72,57,66,68


In [6]:
categorical_cols

,Name,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Joined,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015",...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,"Feb 8, 2018",...,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2
11697,G. Gallon,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,Right,"Jul 1, 2019",...,26+2,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2
11698,J. Flores,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,Right,"Jan 23, 2019",...,44+2,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2
11699,Anderson Silva,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,NaN,...,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0


In [7]:
df.isna().sum()

ID              0
Name            0
Age             0
Nationality     0
Club           18
               ..
CB              0
RCB             0
RB              0
GK              0
OVA             0
Length: 101, dtype: int64

In [8]:
df['Club'] = df['Club'].fillna('Free Agent')
unique_club = df['Club'].value_counts()
unique_club

Everton           35
Crystal Palace    35
OGC Nice          35
Chelsea           33
Sporting CP       33
                  ..
GIF Sundsvall      1
FC Orenburg        1
Germany            1
LKS Lodz           1
Santo André        1
Name: Club, Length: 886, dtype: int64

In [9]:
df.isna().sum().sum()

12099

In [10]:
columns = df.columns
columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Club', 'BP', 'Position',
       'Team & Contract', 'Height', 'Weight',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object', length=101)

In [11]:
for col in df.columns:
    print(col)
    #print(len(df[col].unique()))

ID
Name
Age
Nationality
Club
BP
Position
Team & Contract
Height
Weight
foot
Growth
Joined
Loan Date End
Value
Wage
Release Clause
Contract
Attacking
Crossing
Finishing
Heading Accuracy
Short Passing
Volleys
Skill
Dribbling
Curve
FK Accuracy
Long Passing
Ball Control
Movement
Acceleration
Sprint Speed
Agility
Reactions
Balance
Power
Shot Power
Jumping
Stamina
Strength
Long Shots
Mentality
Aggression
Interceptions
Positioning
Vision
Penalties
Composure
Defending
Marking
Standing Tackle
Sliding Tackle
Goalkeeping
GK Diving
GK Handling
GK Kicking
GK Positioning
GK Reflexes
Total Stats
Base Stats
W/F
SM
A/W
D/W
IR
PAC
SHO
PAS
DRI
DEF
PHY
Hits
LS
ST
RS
LW
LF
CF
RF
RW
LAM
CAM
RAM
LM
LCM
CM
RCM
RM
LWB
LDM
CDM
RDM
RWB
LB
LCB
CB
RCB
RB
GK
OVA


In [12]:
df = df.drop(['Age','Nationality','Club','Team & Contract','Height','Weight','foot','Growth','Joined','Loan Date End','Value','Wage','Release Clause','Contract','W/F','SM','A/W','D/W','Hits','Position','ID','Name'], axis=1)

In [13]:
df.shape

(11701, 79)

In [14]:
df.columns

Index(['BP', 'Attacking', 'Crossing', 'Finishing', 'Heading Accuracy',
       'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve',
       'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'LS',
       'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM',
       'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'GK', 'OVA'],
      dtype='object')

In [15]:
duplicate_rows = df.duplicated()
duplicate_rows.unique()

array([False])

In [16]:
df['IR'] = df['IR'].str.replace(" ★", "")

In [17]:
def remove_plus_and_after(value):
    if isinstance(value, str):
        return value.split('+')[0]
    return value

df = df.applymap(remove_plus_and_after)
df

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CM,258,54,47,43,70,44.0,286,61,44.0,...,59,59,59,58,54,54,54,58,15,64
1,ST,365,66,79,76,68,76.0,375,83,78.0,...,53,53,57,53,48,48,48,53,18,77
2,CAM,336,73,76,34,78,75.0,424,85,89.0,...,56,56,59,53,41,41,41,53,12,80
3,CDM,242,44,42,58,62,36.0,259,54,41.0,...,58,58,56,57,58,58,58,57,14,59
4,CDM,249,49,37,61,68,34.0,280,64,44.0,...,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.0,248,57,49.0,...,60,60,63,63,61,61,61,63,16,65
11697,GK,84,12,14,13,29,16.0,96,17,15.0,...,26,26,25,24,26,26,26,24,69,70
11698,RM,286,64,66,51,64,41.0,291,71,57.0,...,44,44,49,45,35,35,35,45,17,67
11699,CM,254,64,66,51,73,NaN,284,74,NaN,...,68,68,66,64,60,60,60,64,25,68


In [18]:
cols = ['IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'LS','ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM',
'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'GK']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [19]:
df.isna().sum().sum()

580

In [20]:
df.dtypes

BP                  object
Attacking            int64
Crossing             int64
Finishing            int64
Heading Accuracy     int64
                     ...  
CB                   int64
RCB                  int64
RB                   int64
GK                   int64
OVA                  int64
Length: 79, dtype: object

In [21]:
column_means = df.mean()
df = df.fillna(column_means)
df

/var/folders/kd/gp0d9cx1091g1l19n5h72dhr0000gn/T/ipykernel_14042/2185361014.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CM,258,54,47,43,70,44.000000,286,61,44.000000,...,59,59,59,58,54,54,54,58,15,64
1,ST,365,66,79,76,68,76.000000,375,83,78.000000,...,53,53,57,53,48,48,48,53,18,77
2,CAM,336,73,76,34,78,75.000000,424,85,89.000000,...,56,56,59,53,41,41,41,53,12,80
3,CDM,242,44,42,58,62,36.000000,259,54,41.000000,...,58,58,56,57,58,58,58,57,14,59
4,CDM,249,49,37,61,68,34.000000,280,64,44.000000,...,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.000000,248,57,49.000000,...,60,60,63,63,61,61,61,63,16,65
11697,GK,84,12,14,13,29,16.000000,96,17,15.000000,...,26,26,25,24,26,26,26,24,69,70
11698,RM,286,64,66,51,64,41.000000,291,71,57.000000,...,44,44,49,45,35,35,35,45,17,67
11699,CM,254,64,66,51,73,45.067324,284,74,49.617839,...,68,68,66,64,60,60,60,64,25,68


In [22]:
df = df.round(0)
df

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,CM,258,54,47,43,70,44.0,286,61,44.0,...,59,59,59,58,54,54,54,58,15,64
1,ST,365,66,79,76,68,76.0,375,83,78.0,...,53,53,57,53,48,48,48,53,18,77
2,CAM,336,73,76,34,78,75.0,424,85,89.0,...,56,56,59,53,41,41,41,53,12,80
3,CDM,242,44,42,58,62,36.0,259,54,41.0,...,58,58,56,57,58,58,58,57,14,59
4,CDM,249,49,37,61,68,34.0,280,64,44.0,...,64,64,64,63,61,61,61,63,15,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.0,248,57,49.0,...,60,60,63,63,61,61,61,63,16,65
11697,GK,84,12,14,13,29,16.0,96,17,15.0,...,26,26,25,24,26,26,26,24,69,70
11698,RM,286,64,66,51,64,41.0,291,71,57.0,...,44,44,49,45,35,35,35,45,17,67
11699,CM,254,64,66,51,73,45.0,284,74,50.0,...,68,68,66,64,60,60,60,64,25,68


In [23]:
X = df.drop(['OVA'], axis=1)
X

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,CM,258,54,47,43,70,44.0,286,61,44.0,...,59,59,59,59,58,54,54,54,58,15
1,ST,365,66,79,76,68,76.0,375,83,78.0,...,53,53,53,57,53,48,48,48,53,18
2,CAM,336,73,76,34,78,75.0,424,85,89.0,...,56,56,56,59,53,41,41,41,53,12
3,CDM,242,44,42,58,62,36.0,259,54,41.0,...,58,58,58,56,57,58,58,58,57,14
4,CDM,249,49,37,61,68,34.0,280,64,44.0,...,64,64,64,64,63,61,61,61,63,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,LB,224,62,24,51,59,28.0,248,57,49.0,...,60,60,60,63,63,61,61,61,63,16
11697,GK,84,12,14,13,29,16.0,96,17,15.0,...,26,26,26,25,24,26,26,26,24,69
11698,RM,286,64,66,51,64,41.0,291,71,57.0,...,44,44,44,49,45,35,35,35,45,17
11699,CM,254,64,66,51,73,45.0,284,74,50.0,...,68,68,68,66,64,60,60,60,64,25


In [24]:
y = df['OVA']
y

0        64
1        77
2        80
3        59
4        65
         ..
11696    65
11697    70
11698    67
11699    68
11700    64
Name: OVA, Length: 11701, dtype: int64

In [25]:
X_cat = df.select_dtypes('object')
X_num = X._get_numeric_data() 

In [26]:
X_cat

,BP
0,CM
1,ST
2,CAM
3,CDM
4,CDM
...,...
11696,LB
11697,GK
11698,RM
11699,CM


In [27]:
X_num

,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,258,54,47,43,70,44.0,286,61,44.0,55,...,59,59,59,59,58,54,54,54,58,15
1,365,66,79,76,68,76.0,375,83,78.0,72,...,53,53,53,57,53,48,48,48,53,18
2,336,73,76,34,78,75.0,424,85,89.0,91,...,56,56,56,59,53,41,41,41,53,12
3,242,44,42,58,62,36.0,259,54,41.0,46,...,58,58,58,56,57,58,58,58,57,14
4,249,49,37,61,68,34.0,280,64,44.0,45,...,64,64,64,64,63,61,61,61,63,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,224,62,24,51,59,28.0,248,57,49.0,29,...,60,60,60,63,63,61,61,61,63,16
11697,84,12,14,13,29,16.0,96,17,15.0,19,...,26,26,26,25,24,26,26,26,24,69
11698,286,64,66,51,64,41.0,291,71,57.0,38,...,44,44,44,49,45,35,35,35,45,17
11699,254,64,66,51,73,45.0,284,74,50.0,63,...,68,68,68,66,64,60,60,60,64,25


In [28]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_cat)

cols = encoder.get_feature_names_out(input_features=X_cat.columns)

X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)

X_cat_encode.head()



,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,BP_RM,BP_RW,BP_RWB,BP_ST
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
pickle.dump(encoder, open('encoder.pkl', 'wb'))

# Normalizing the data

In [30]:
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different

transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
X_num_normal = pd.DataFrame(x_normalized, columns=X_num.columns)

(11701, 77)


In [31]:
pickle.dump(transformer, open('transformer.pkl', 'wb'))

In [32]:
X = pd.concat([X_cat_encode, X_num_normal], axis=1)
X.head()

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.614286,0.614286,0.614286,0.625000,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.528571,0.528571,0.528571,0.597222,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.571429,0.571429,0.571429,0.625000,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.600000,0.600000,0.600000,0.583333,0.623188,0.583333,0.583333,0.583333,0.623188,0.063291
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.685714,0.685714,0.685714,0.694444,0.710145,0.625000,0.625000,0.625000,0.710145,0.075949


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9360, 91)
(2341, 91)
(9360,)
(2341,)


In [34]:
X_train.head()

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
1775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.342857,0.342857,0.342857,0.375000,0.362319,0.277778,0.277778,0.277778,0.362319,0.101266
2455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.471429,0.471429,0.471429,0.416667,0.391304,0.402778,0.402778,0.402778,0.391304,0.113924
93,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.700000,0.700000,0.700000,0.666667,0.710145,0.722222,0.722222,0.722222,0.710145,0.113924
8952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.471429,0.471429,0.471429,0.472222,0.478261,0.402778,0.402778,0.402778,0.478261,0.050633
7810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.685714,0.685714,0.685714,0.694444,0.695652,0.597222,0.597222,0.597222,0.695652,0.101266


In [35]:
y_train.head()

1775    62
2455    74
93      69
8952    55
7810    79
Name: OVA, dtype: int64

In [36]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [37]:
pickle.dump(lm, open('lm.pkl', 'wb'))

In [38]:
y = df['OVA']
y.head()

0    64
1    77
2    80
3    59
4    65
Name: OVA, dtype: int64

In [39]:
y_train.head()

1775    62
2455    74
93      69
8952    55
7810    79
Name: OVA, dtype: int64

In [40]:
correlations_matrix = X.corr()
correlations_matrix.shape
correlations_matrix

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
BP_CB,1.000000,-0.132272,-0.033385,-0.116474,-0.152234,-0.114111,-0.106877,-0.054006,-0.058281,-0.113825,...,0.273237,0.273237,0.273237,0.174758,0.258941,0.441949,0.441949,0.441949,0.258941,-0.151161
BP_CDM,-0.132272,1.000000,-0.019395,-0.067667,-0.088442,-0.066294,-0.062092,-0.031376,-0.033859,-0.066128,...,0.273379,0.273379,0.273379,0.221203,0.229984,0.256568,0.256568,0.256568,0.229984,-0.079896
BP_CF,-0.033385,-0.019395,1.000000,-0.017079,-0.022322,-0.016732,-0.015672,-0.007919,-0.008546,-0.016690,...,0.009861,0.009861,0.009861,0.012951,0.000758,-0.015388,-0.015388,-0.015388,0.000758,-0.010894
BP_CM,-0.116474,-0.067667,-0.017079,1.000000,-0.077879,-0.058376,-0.054676,-0.027628,-0.029815,-0.058230,...,0.214316,0.214316,0.214316,0.182647,0.174822,0.163729,0.163729,0.163729,0.174822,-0.068161
BP_GK,-0.152234,-0.088442,-0.022322,-0.077879,1.000000,-0.076299,-0.071462,-0.036111,-0.038969,-0.076107,...,-0.677935,-0.677935,-0.677935,-0.747206,-0.719274,-0.612954,-0.612954,-0.612954,-0.719274,0.978290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LCB,0.441949,0.256568,-0.015388,0.163729,-0.612954,0.163396,-0.067902,-0.074689,0.085449,0.178632,...,0.958695,0.958695,0.958695,0.909470,0.949929,1.000000,1.000000,1.000000,0.949929,-0.564154
CB,0.441949,0.256568,-0.015388,0.163729,-0.612954,0.163396,-0.067902,-0.074689,0.085449,0.178632,...,0.958695,0.958695,0.958695,0.909470,0.949929,1.000000,1.000000,1.000000,0.949929,-0.564154
RCB,0.441949,0.256568,-0.015388,0.163729,-0.612954,0.163396,-0.067902,-0.074689,0.085449,0.178632,...,0.958695,0.958695,0.958695,0.909470,0.949929,1.000000,1.000000,1.000000,0.949929,-0.564154
RB,0.258941,0.229984,0.000758,0.174822,-0.719274,0.189893,0.011262,-0.030588,0.107127,0.203176,...,0.980695,0.980695,0.980695,0.991403,1.000000,0.949929,0.949929,0.949929,1.000000,-0.666262


In [41]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)



0.9090642437868288

In [42]:
#the last thing should be create a def with all the code we used


In [43]:
def datacleaner(df):
    
    df['Club'] = df['Club'].fillna('Free Agent')
    df = df.drop(['Age','Nationality','Club','Team & Contract','Height','Weight','foot','Growth','Joined','Loan Date End','Value','Wage','Release Clause','Contract','W/F','SM','A/W','D/W','Hits','Position','ID','Name'], axis=1)
    df['IR'] = df['IR'].str.replace(" ★", "")
    def remove_plus_and_after(value):
        if isinstance(value, str):
            return value.split('+')[0]
            return value
    cols = ['IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'LS','ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM','LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB','LCB', 'CB', 'RCB', 'RB', 'GK']
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    column_means = df.mean()
    df = df.fillna(column_means)
    X = df.drop(['OVA'], axis=1)
    y = df['OVA']
    X_cat = df.select_dtypes('object')
    X_num = X._get_numeric_data()
    from sklearn.preprocessing import OneHotEncoder
    encoder = OneHotEncoder(drop='first').fit(X_cat)
    cols = encoder.get_feature_names_out(input_features=X_cat.columns)
    X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)
    from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
    transformer = MinMaxScaler().fit(X_num)
    x_normalized = transformer.transform(X_num)
    print(x_normalized.shape)
    X_num_normal = pd.DataFrame(x_normalized, columns=X_num.columns)
    X = pd.concat([X_cat_encode, X_num_normal], axis=1)
    
    return df 

X

,BP_CB,BP_CDM,BP_CF,BP_CM,BP_GK,BP_LB,BP_LM,BP_LW,BP_LWB,BP_RB,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.614286,0.614286,0.614286,0.625000,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.528571,0.528571,0.528571,0.597222,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.571429,0.571429,0.571429,0.625000,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.600000,0.600000,0.600000,0.583333,0.623188,0.583333,0.583333,0.583333,0.623188,0.063291
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.685714,0.685714,0.685714,0.694444,0.710145,0.625000,0.625000,0.625000,0.710145,0.075949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.628571,0.628571,0.628571,0.680556,0.710145,0.625000,0.625000,0.625000,0.710145,0.088608
11697,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.142857,0.142857,0.142857,0.152778,0.144928,0.138889,0.138889,0.138889,0.144928,0.759494
11698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.400000,0.400000,0.400000,0.486111,0.449275,0.263889,0.263889,0.263889,0.449275,0.101266
11699,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.742857,0.742857,0.742857,0.722222,0.724638,0.611111,0.611111,0.611111,0.724638,0.202532


In [44]:
# Function ready to use!

In [45]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

0.906441302390728

In [46]:
y_test[:5]

2405     49
8025     70
10025    62
10933    72
1402     64
Name: OVA, dtype: int64

In [47]:
predictions_test[:5]

array([50.25669098, 69.6998291 , 62.57473755, 70.7089386 , 65.58612061])

In [48]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test,predictions_test)
mse

4.430055865828495

In [49]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_test))
rmse

2.1047697892711437

In [50]:
y_test.mean()

66.57624946604015

In [51]:
selected_rows = df.loc[[2405, 8025, 10025,10933,1402]]
selected_rows

,BP,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
2405,RB,166,40,22,44,31,29.0,161,53,26.0,...,42,42,47,47,47,47,47,47,15,49
8025,CAM,323,72,58,50,72,71.0,364,73,76.0,...,49,49,50,46,40,40,40,46,16,70
10025,ST,286,58,60,54,57,57.0,279,63,60.0,...,46,46,50,48,44,44,44,48,15,62
10933,CAM,313,69,59,38,76,71.0,367,72,77.0,...,68,68,67,65,59,59,59,65,18,72
1402,RM,274,56,51,59,58,50.0,263,61,39.0,...,50,50,53,51,46,46,46,51,16,64


In [52]:
from sklearn.metrics import mean_absolute_error, r2_score

In [53]:
mae = mean_absolute_error(y_test, predictions_test)
print(mae)

1.6325837517843853


In [54]:
rmse = math.sqrt(mse)
print(rmse)

2.1047697892711437


In [55]:
r2 = r2_score(y_test, predictions_test)
r2

0.906441302390728

In [56]:
# Here we look at the results of both the training and the testing

# predictions_test = lm.predict(X_test)
# r2_score(y_test, predictions_test)

# r2 = r2_score(y_test, predictions_test)
# r2

In [57]:
# def datacleaner(df):
    
#     df['Club'] = df['Club'].fillna('Free Agent')
#     df = df.drop(['Age','Nationality','Club','Team & Contract','Height','Weight','foot','Growth','Joined','Loan Date End','Value','Wage','Release Clause','Contract','W/F','SM','A/W','D/W','Hits','Position','ID','Name'], axis=1)
#     df['IR'] = df['IR'].str.replace(" ★", "")
#     def remove_plus_and_after(value):
#          if isinstance(value, str):
#             return value.split('+')[0]
#          return value
#     df = df.applymap(remove_plus_and_after)
#     cols = ['IR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'LS','ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM','LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB','LCB', 'CB', 'RCB', 'RB', 'GK']
#     df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
#     column_means = df.mean()
#     df = df.fillna(column_means)
    
#     return df

# modified_df = datacleaner(df)

In [58]:
# modified_df.isna().sum()

In [59]:
# modified_df

In [60]:
# X = modified_df.drop(['OVA'], axis=1)
# y2 = modified_df['OVA']
# X_cat = modified_df.select_dtypes('object')
# X_num = X._get_numeric_data()
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(drop='first').fit(X_cat)
# cols = encoder.get_feature_names_out(input_features=X_cat.columns)
# X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)
# from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
# transformer = MinMaxScaler().fit(X_num)
# x_normalized = transformer.transform(X_num)
# print(x_normalized.shape)
# X_num_normal = pd.DataFrame(x_normalized, columns=X_num.columns)
# X = pd.concat([X_cat_encode, X_num_normal], axis=1)

In [61]:
# X

In [62]:
# y2

In [63]:
# y

In [64]:
# use pickle dump #with changes!